In [58]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [59]:
data = pd.read_csv("IMDB Dataset.csv")
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [60]:
data.shape

(50000, 2)

In [76]:
print('Class distribution: ', data['sentiment'].value_counts())

Class distribution:  sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [61]:
print(data.isnull().sum())

review       0
sentiment    0
dtype: int64


In [62]:
print(data.nunique())

review       49582
sentiment        2
dtype: int64


In [63]:
import re
from nltk.corpus import stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load spacy model once (not inside function)
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not isinstance(text, str):
        return "" 
    
    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r"[^a-z\s]", "", text)
    
    text = text.strip()
    
    # Split and remove stopwords & empty tokens
    words = [w for w in text.split() if w and w not in STOP_WORDS]
    
    # Lemmatize
    doc = nlp(" ".join(words))
    lemmas = [token.lemma_ for token in doc if token.text.strip()]
    
    
    return " ".join(lemmas)


In [64]:
first_20_rev = data[:20].copy()


In [65]:
first_20_rev["clean_review"] = first_20_rev["review"].apply(preprocess_text)

first_20_rev

,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode you ll hook ...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,negative,basically there s family little boy jake think...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
5,"Probably my all-time favorite movie, a story o...",positive,probably alltime favorite movie story selfless...
6,I sure would like to see a resurrection of a u...,positive,sure like resurrection date seahunt series tec...
7,"This show was an amazing, fresh & innovative i...",negative,amazing fresh innovative idea s air year brill...
8,Encouraged by the positive comments about this...,negative,encourage positive comment film look forward w...
9,If you like original gut wrenching laughter yo...,positive,like original gut wrench laughter like movie y...


In [66]:
data["clean_review"] = data["review"].apply(preprocess_text)

data.head()

,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode you ll hook ...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,negative,basically there s family little boy jake think...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

tfidf = TfidfVectorizer(max_features=5000)
le = LabelEncoder()

y = le.fit_transform(data['sentiment'])

X = tfidf.fit_transform(data['clean_review'])



In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix


nb_model = MultinomialNB(alpha=0.5)
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Naive Bayes Confusion Matrix")
plt.show()


NameError: name 'X_train' is not defined